In [1]:
import os
import pandas as pd
import json

from dict_process import *

sheets = ['传送模设计科', '设计二科', '设计一科', '研磨', '车铣钻', 'CNC', '线割组']


In [3]:
# 读取字典
with open('dict/touse.json', 'r') as f:
    errors = json.load(f)

path = '/home/keae/Mould/data'
filelist = os.listdir(path)
filelist.sort()

print('识别错误')
for file in filelist[1:4]:
    print(os.path.join(path, file))
    dfs = pd.read_excel(os.path.join(path, file), header=1, sheet_name=None)

    writer = pd.ExcelWriter('results/'+file+'.xls')
    for sheet in sheets[0:1]:
        df = dfs[sheet]
        df = df[['异常单编码', '品管异常描述']]
        df = df.replace('\n', '', regex=True)
        df[['错误位置', '错误类型', '原文错误']] = pd.DataFrame([['', '', '']], index=df.index)

        dfs[sheet] = recog(df, errors[sheet])
        dfs[sheet].to_excel(writer, sheet)

    writer.save()


    异常单编码                                             品管异常描述 错误位置 错误类型 原文错误
0   51389                 MGJ033-0106-0606-P1858压块设计未下图（未备料）               
1   51379  GL001-0202-P5821，P5822，P5823，刀口没有斜度.设计未标注，导致没加工斜度               
2   51329  MGJ033-0106-p1809A 平1812A 冲头与刀口对不上.3D的刀口处有个台阶。...               
3   51285                     GL001-0202-P7804镶块高度低于P7500镶块面               
4   51265                   标准件LY037-0205-S2018 无避位 （2件）设计错误               
5   51185             GL003-0202-P6008，因加工让位不够导致崩裂，烧焊返修后，新制。               
6   51146                       GL001-0202-P4502/P4503 设计漏下图               
7   51124  GL007-0202-P4077 P4078压板与GL007-0202-P4042上模导柱定...               
8   51122      CEN140-0707-P207A上模刀块。图档图层线是3D线，无法分解。此问题需要根治。               
9   51103                              GL001-0202-S4030 设计买错               
10  51085  GL001-0202-P7810 P7817 设计出错图，钳工要求设计出图软料新制不加工螺丝...               
11  51047                   MPC045-0102-P6809 设计出垫片不够导致加工量不足               
12  51031   

In [4]:
path = '/home/keae/Mould/data'
filelist = os.listdir(path)
filelist.sort()

errors = {}
print('生成字典：')
for file in filelist[1:3]:
    print(os.path.join(path, file), end=': ')

    df = pd.read_excel(os.path.join(path, file), header=1, sheet_name=None)
    for sheet in sheets[0:1]:
        print(sheet, end=',')
        tmp = df[sheet]
        tmp = tmp[['异常单编码', '品管异常描述', '错误位置1', '错误位置2', '原错误类型', '归纳错误类型']]
        tmp = tmp.replace('\n', '', regex=True)

        if sheet in errors:
            error = errors[sheet]
        else:
            error = {'错误位置': [], '错误类型': {}}

        errors[sheet] = gener(tmp, error)

    print()


with open('dict/extract.json', 'w') as f:
    json.dump(errors, f, sort_keys=True, indent=4, ensure_ascii=False)

# print(errors)


生成字典：
/home/keae/Mould/data/9.11-9.17.xls: 传送模设计科,
/home/keae/Mould/data/9.18-9.24.xls: 传送模设计科,


In [6]:
with open('dict/extract.json', 'r') as f:
    extract = json.load(f)
with open('dict/expert.json', 'r') as f:
    expert = json.load(f)

d = {}
for sheet in sheets[0:1]:
    d[sheet] = merge(extract[sheet], expert[sheet])


with open('dict/merge.json', 'w') as f:
    json.dump(d, f, sort_keys=True, indent=4, ensure_ascii=False)


In [2]:
with open('dict/merge.json', 'r') as f:
    merge = json.load(f)

for sheet in sheets[0:1]:
    merge[sheet]['错误类型']=rever(merge[sheet]['错误类型'])


with open('dict/touse.json', 'w') as f:
    json.dump(merge, f, sort_keys=True, indent=4, ensure_ascii=False)


{'不够': ['不够'], '加工错误': ['未打点', '少加工', '与图纸不符', '加工不到位'], '对不上': ['对不上', '对应不上', '孔位不对'], '少画': ['少画'], '尺寸不匹配': ['尺寸不匹配'], '干涉': ['有干涉'], '无法分解': ['无法分解'], '无避位': ['无避位'], '有干涉': ['有干涉', '干涉'], '未标注': ['未标注'], '标注错误': ['标注错误'], '漏下图': ['未下图', '漏下图'], '漏标注': ['漏标注'], '让位不够': ['让位不够'], '设计买错': ['设计买错'], '设计错误': ['出错图', '设计错误', '设计有避空', '设计有误', '漏画销钉孔', '避空'], '钳工丢失': ['钳工丢失'], '高度不匹配': ['高度低于']}
{'不够': '不够', '未打点': '加工错误', '少加工': '加工错误', '与图纸不符': '加工错误', '加工不到位': '加工错误', '对不上': '对不上', '对应不上': '对不上', '孔位不对': '对不上', '少画': '少画', '尺寸不匹配': '尺寸不匹配', '有干涉': '有干涉', '无法分解': '无法分解', '无避位': '无避位', '干涉': '有干涉', '未标注': '未标注', '标注错误': '标注错误', '未下图': '漏下图', '漏下图': '漏下图', '漏标注': '漏标注', '让位不够': '让位不够', '设计买错': '设计买错', '出错图': '设计错误', '设计错误': '设计错误', '设计有避空': '设计错误', '设计有误': '设计错误', '漏画销钉孔': '设计错误', '避空': '设计错误', '钳工丢失': '钳工丢失', '高度低于': '高度不匹配'}
